# Language Model 

adopted from https://github.com/harvard-ml-courses/cs287-s18/blob/master/HW2/Homework%202.ipynb

We will use [torchtext](https://github.com/pytorch/text) to create vocabulary, and load datasets into batches. Please refer to their GitHub README page for a quick tutorial. 

In [3]:
import torchtext
from torchtext.vocab import Vectors
import torch
import numpy as np
import random

USE_CUDA = torch.cuda.is_available()

random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)

    
BATCH_SIZE = 32
EMBEDDING_SIZE = 650
MAX_VOCAB_SIZE = 50000
LOG_FILE = "language-model.log"

In [4]:
TEXT = torchtext.data.Field(lower=True)
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(path=".", 
    train="text8.train.txt", validation="text8.dev.txt", test="text8.test.txt", text_field=TEXT)
TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)
print("vocabulary size: {}".format(len(TEXT.vocab)))

VOCAB_SIZE = len(TEXT.vocab)
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=BATCH_SIZE, device=-1, bptt_len=32, repeat=False)


vocabulary size: 50002


How to get text and target using torchtext. The learning goal of our language model is to predict the next word using the previous words. 

In [12]:
it = iter(train_iter)
batch = next(it)
print(" ".join([TEXT.vocab.itos[i] for i in batch.text[:,1].data]))
print(" ".join([TEXT.vocab.itos[i] for i in batch.target[:,1].data]))

had dropped to just three zero zero zero k it was then cool enough to allow the nuclei to capture electrons this process is called recombination during which the first neutral atoms
dropped to just three zero zero zero k it was then cool enough to allow the nuclei to capture electrons this process is called recombination during which the first neutral atoms took


### Define the model

In [6]:
import torch
import torch.nn as nn


class RNNModel(nn.Module):
    """ Container module with an encoder, a recurrent module, and a decoder.
        Feel free to add more methods into this class if necessary. 
    """

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        ''' Create the layers of your model. You will need the following layers:
            - embedding layer
            - recurrent neural network layer (LSTM, GRU)
            - linear decoding layer to map from hidden vector to the vocabulary
            - optionally, add the dropout layer. The dropout layer can be put after 
                the embedding layer or/and after the RNN layer, or other places. 
            - optionally, initialize your model parameters. Look for papers/blogs 
                online for good parameter initialization methods. 
            Please read the documentation for how to build LSTM with PyTorch. 
            https://pytorch.org/docs/stable/nn.html?highlight=lstm#torch.nn.LSTM
        '''
        super(RNNModel, self).__init__()
        # TODO
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        ''' The forward layer. You will need to do the following:
            - embed word index to word vectors
            - run RNN
            - linear layer to decode hidden vectors to output words
        '''
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz, requires_grad=True):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad),
                    weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad))
        else:
            return weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad)

Implement the evaluation script, return the loss. 

In [7]:
def evaluate(model, data):
    model.eval()
    total_loss = 0.
    it = iter(data)
    total_count = 0.
    with torch.no_grad():
        ''' Fill in your evaluation code here. The evaulation follows the same logic as training. 
        You might want to finish the training part first. 
        '''
        # TODO 
        
        hidden = model.init_hidden(BATCH_SIZE, requires_grad=False)
        for i, batch in enumerate(it):
            data, target = batch.text, batch.target
            if USE_CUDA:
                data, target = data.cuda(), target.cuda()
            hidden = repackage_hidden(hidden)
            with torch.no_grad():
                output, hidden = model(data, hidden)
            loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1))
            total_count += np.multiply(*data.size())
            total_loss += loss.item()*np.multiply(*data.size())
            
    loss = total_loss / total_count
    model.train()
    return loss


In [8]:
import copy
GRAD_CLIP = 1.
NUM_EPOCHS = 10

# Remove this part
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

model = RNNModel("LSTM", VOCAB_SIZE, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)
if USE_CUDA:
    model = model.cuda()

loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
val_losses = []
for epoch in range(NUM_EPOCHS):
    model.train()
    it = iter(train_iter)
#     hidden = model.init_hidden(BATCH_SIZE)
    for i, batch in enumerate(it):
        ''' The training code. You need to do the following:
            - prepare the training tensors, including the tensors of the history words and the predicted words
            - zero the model gradidents
            - predict the next words using the model
            - compute the cross entropy loss
            - do back propagation 
            - clip the gradients as we are training RNN
            - update the model with the gradidents
            - optionally, print out the batch loss after every 1000 iterations. 
            - optionally, evaluate your model on DEV after every 10000 iterations and save it to best_model. 
        '''
        # TODO
        
        data, target = batch.text, batch.target
        if USE_CUDA:
            data, target = data.cuda(), target.cuda()
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1))
        loss.backward()
        # apply gradient clipping to prevent the exploding gradient problem in RNN
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        optimizer.step()
        if i % 1000 == 0:
            print("epoch", epoch, "iter", i, "loss", loss.item())
    
        if i % 10000 == 0:
            val_loss = evaluate(model, val_iter)
            with open(LOG_FILE, "a") as fout:
                print("epoch: {}, iteration: {}, perplexity: {}".format(epoch, i, np.exp(val_loss)))
                fout.write("epoch: {}, iteration: {}, perplexity: {}\n".format(epoch, i, np.exp(val_loss)))
            
            if len(val_losses) == 0 or val_loss < min(val_losses):
                print("best model, val loss: ", val_loss)
                best_model = copy.deepcopy(model)
                with open("lm-best.th", "wb") as fout:
                    torch.save(best_model.state_dict(), fout)
            else:
                learning_rate /= 4.
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            val_losses.append(val_loss)

epoch 0 iter 0 loss 10.821989059448242


/Users/zeweichu/anaconda3/envs/py36/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


KeyboardInterrupt: 

In [ ]:
val_loss = evaluate(best_model, val_iter)
print("perplexity: ", np.exp(val_loss))

### Use the best model to evaluate the test dataset. 

In [ ]:
test_loss = evaluate(best_model, test_iter)
print("perplexity: ", np.exp(test_loss))

Generate some sentences. 

In [ ]:
hidden = best_model.init_hidden(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input = torch.randint(VOCAB_SIZE, (1, 1), dtype=torch.long).to(device)
words = []
for i in range(100):
    output, hidden = best_model(input, hidden)
    word_weights = output.squeeze().exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    input.fill_(word_idx)
    word = TEXT.vocab.itos[word_idx]
    words.append(word)
print(" ".join(words))